In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras import layers 
from tensorflow.keras import Model
from tensorflow.keras.applications.xception import Xception
from keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
pip list | grep tensorflow

In [2]:
IMAGE_SIZE = (128, 128)
INPUT_SHAPE = IMAGE_SIZE + (3,)
VALIDATION_SPLIT = 0.2
BATCH_SIZE = 32

In [7]:
BUCKET = 9

gdrive_dir = "/content/drive/MyDrive"
working_dir = os.path.join(gdrive_dir, "CS3244 Project")
data_dir = os.path.join(working_dir, "landmarks/international/data_split", str(BUCKET))
model_root_dir = os.path.join(working_dir, "models/Xihao")

In [8]:
num_of_labels = len(os.listdir(data_dir))

print('number of labels:', num_of_labels)

number of labels: 200


In [9]:
# dataflow_kwargs = dict(target_size=IMAGE_SIZE, batch_size=BATCH_SIZE, interpolation="bilinear")

train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale = 1./255, 
    validation_split = VALIDATION_SPLIT,
    rotation_range = 30,
    width_shift_range = 0.1,
    height_shift_range = 0.1,
    shear_range = 0.1,
    zoom_range = 0.1,
    brightness_range = [0.9,1.1],
    fill_mode = 'nearest'
)
train_generator = train_datagen.flow_from_directory(
    data_dir,
    subset = "training", 
    shuffle = True, 
    target_size = IMAGE_SIZE ,
    batch_size = BATCH_SIZE,
    class_mode = 'categorical',
)

validation_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split = VALIDATION_SPLIT
)
validation_generator = validation_datagen.flow_from_directory(
    data_dir,
    subset = "validation", 
    shuffle = False, 
    target_size = IMAGE_SIZE,
    batch_size = BATCH_SIZE,
    class_mode = 'categorical'
)

Found 1371 images belonging to 200 classes.
Found 249 images belonging to 200 classes.


In [ ]:
base_model = Xception(
    input_shape = INPUT_SHAPE,
    include_top = False,
    weights='imagenet'
)

83689472/83683744 [==============================] - 1s 0us/step


In [ ]:
base_model.summary()

In [ ]:
last_layer = base_model.get_layer("block14_sepconv2_act")
last_output = last_layer.output

In [ ]:
last_output_count = 512

x = layers.Flatten()(last_output)
x = layers.Dense(1024, activation='relu')(x)
x = layers.Dropout(0.2)(x)
x = layers.Dense(256, activation='relu')(x)
x = layers.BatchNormalization(momentum=0.95)(x)
x = layers.Dense(num_of_labels, activation='softmax')(x)

model = Model(base_model.input, x)

In [10]:
model_version = 8

load_model_name = "xception_international_base_normalised_adam_2_" + str(model_version)
load_model_dir = os.path.join(model_root_dir, load_model_name)

model = tf.keras.models.load_model(load_model_dir)

In [19]:
model.compile(
  loss = 'categorical_crossentropy',
  # optimizer = tf.keras.optimizers.RMSprop(lr=0.001),
  optimizer = tf.keras.optimizers.SGD(learning_rate=0.001),
  # optimizer = tf.keras.optimizers.Adam(learning_rate=0.001),   
  metrics = ['accuracy']
)

In [ ]:
model.summary()

In [17]:
steps_per_epoch = int(train_generator.samples / BATCH_SIZE)
validation_steps = int(validation_generator.samples / BATCH_SIZE)

print("Steps per epoch:", steps_per_epoch)
print("Validation steps:", validation_steps)

Steps per epoch: 42
Validation steps: 7


In [ ]:
history = model.fit(
  train_generator,
  validation_data = validation_generator,
  epochs = 15,
  steps_per_epoch = steps_per_epoch,
  validation_steps = validation_steps
)


save_model_name = "xception_international_base_normalised_adam_2_" + str(BUCKET)
save_model_dir = os.path.join(model_root_dir, save_model_name)
tf.keras.models.save_model(model, save_model_dir)


acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'r', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend(loc=0)
plt.figure()

plt.plot(epochs, loss, 'r', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')

df = pd.DataFrame(history.history)
hist_dir = os.path.join(model_root_dir, "history/" + save_model_name+".csv")
df.to_csv(hist_dir)

In [ ]:
df = pd.DataFrame(history.history)
hist_dir = os.path.join(model_root_dir, "history/" + save_model_name+".csv")
df.to_csv(hist_dir)